# NLP Revision

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]='3'
import tensorflow as tf
from helper_functions import importTensorflow
importTensorflow()

2.15.0
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050 Laptop GPU, compute capability 8.6


In [2]:
from helper_functions import plot_loss_curves, compare_historys

In [3]:
import pandas as pd, numpy as np
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
#Shuffle train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [5]:
len(train_df), len(test_df)

(7613, 3263)

In [6]:
# Split data
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                           train_df_shuffled.target.to_numpy(),
                                                                           test_size=0.1,
                                                                           random_state=42)

In [7]:
# Text vectorization
#default
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=None,
                                                    standardize="lower_and_strip_punctuation",
                                                    split='whitespace',
                                                    ngrams=None,
                                                    output_mode='int',
                                                    output_sequence_length=None,
                                                    pad_to_max_tokens=False)

In [8]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [9]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                    output_mode='int',
                                                    output_sequence_length=max_length)
#fit the text vectorizer
text_vectorizer.adapt(train_sentences)
text_vectorizer(["Hi there, how are you"])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1663,   74,   62,   22,   12,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [10]:
#embedding
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                      output_dim=128,
                                      input_length=max_length)
embedding

In [11]:
embedding(text_vectorizer(["Hi there, how are you"]))

<tf.Tensor: shape=(1, 15, 128), dtype=float16, numpy=
array([[[ 4.3884e-02,  3.0155e-03, -7.4883e-03, ..., -2.1255e-02,
          4.4800e-02,  1.2260e-02],
        [ 5.8830e-05,  8.0185e-03,  2.5528e-02, ...,  4.2877e-02,
         -2.9785e-02,  1.4481e-02],
        [ 1.9760e-03, -2.9144e-02, -1.2077e-02, ...,  3.4729e-02,
         -3.2043e-02, -1.9592e-02],
        ...,
        [-3.3081e-02,  2.3666e-02,  2.8915e-02, ...,  3.9276e-02,
         -2.4918e-02,  1.3336e-02],
        [-3.3081e-02,  2.3666e-02,  2.8915e-02, ...,  3.9276e-02,
         -2.4918e-02,  1.3336e-02],
        [-3.3081e-02,  2.3666e-02,  2.8915e-02, ...,  3.9276e-02,
         -2.4918e-02,  1.3336e-02]]], dtype=float16)>

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

baseline_score = model_0.score(val_sentences, val_labels)
baseline_preds = model_0.predict(val_sentences)

from helper_functions import calculate_results
calculate_results(val_labels, baseline_preds)

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [13]:
#model 1 dense model
inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)(x)
model_1 = tf.keras.Model(inputs, outputs)
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
history_1 = model_1.fit(train_sentences, train_labels,
                        validation_data = (val_sentences, val_labels),
                        epochs=5)

model_1_preds = tf.squeeze(tf.round(model_1.predict(val_sentences)))
calculate_results(val_labels, model_1_preds)

Epoch 1/5
  1/215 [..............................] - ETA: 6:52 - loss: 0.6929 - accuracy: 0.5000

I0000 00:00:1703410303.801683  140566 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


215/215 [==============================] - 8s 26ms/step - loss: 0.6124 - accuracy: 0.6941 - val_loss: 0.5381 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4424 - accuracy: 0.8162 - val_loss: 0.4684 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 2s 12ms/step - loss: 0.3481 - accuracy: 0.8584 - val_loss: 0.4571 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2853 - accuracy: 0.8901 - val_loss: 0.4708 - val_accuracy: 0.7940
Epoch 5/5
24/24 [==============================] - 0s 2ms/step


{'accuracy': 78.21522309711287,
 'precision': 0.7853104900348995,
 'recall': 0.7821522309711286,
 'f1': 0.7796592181651525}

In [14]:
#model2 - LSTM
inputs2 = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs2)
x = embedding(x)
x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x)
# x = tf.keras.layers.Dense(64, activation=tf.keras.activations.relu)(x)
outputs2 = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)(x)
model_2 = tf.keras.Model(inputs2, outputs2)
model_2.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
history_2 = model_2.fit(train_sentences, train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels))
model_2_preds = model_2.predict(val_sentences)
calculate_results(val_labels, tf.squeeze(tf.round(model_2_preds)))

Epoch 1/5
215/215 [==============================] - 11s 39ms/step - loss: 0.2281 - accuracy: 0.9200 - val_loss: 0.5390 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.1566 - accuracy: 0.9419 - val_loss: 0.6545 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.1285 - accuracy: 0.9520 - val_loss: 0.6846 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1038 - accuracy: 0.9580 - val_loss: 0.9048 - val_accuracy: 0.7730
Epoch 5/5
24/24 [==============================] - 1s 6ms/step


{'accuracy': 77.29658792650919,
 'precision': 0.7747736761679446,
 'recall': 0.7729658792650919,
 'f1': 0.7708425630155245}

In [15]:
#model3 - GRU
inputs3 = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs3)
x = embedding(x)
x = tf.keras.layers.GRU(64, return_sequences=True)(x)
x = tf.keras.layers.GRU(64)(x)
outputs3 = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)(x)
model_3 = tf.keras.Model(inputs3, outputs3)
model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
history_3 = model_3.fit(train_sentences, train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels))
mdoel_3_preds = model_3.predict(val_sentences)
calculate_results(val_labels, tf.squeeze(tf.round(mdoel_3_preds)))

Epoch 1/5
215/215 [==============================] - 11s 40ms/step - loss: 0.1494 - accuracy: 0.9454 - val_loss: 0.7316 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0864 - accuracy: 0.9677 - val_loss: 0.6558 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0689 - accuracy: 0.9720 - val_loss: 1.1077 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0559 - accuracy: 0.9752 - val_loss: 0.9254 - val_accuracy: 0.7677
Epoch 5/5
24/24 [==============================] - 0s 5ms/step


{'accuracy': 77.82152230971128,
 'precision': 0.7894499714481796,
 'recall': 0.7782152230971129,
 'f1': 0.7729173197612593}

In [16]:
#model4 - Bidirectional
inputs4 = tf.keras.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs4)
x = embedding(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64))(x)
outputs4 = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)(x)
model_4 = tf.keras.Model(inputs4, outputs4)
model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
history_4 = model_4.fit(train_sentences, train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels))
model_4_preds = model_4.predict(val_sentences)
calculate_results(val_labels, tf.squeeze(tf.round(model_4_preds)))

Epoch 1/5
215/215 [==============================] - 15s 49ms/step - loss: 0.1024 - accuracy: 0.9686 - val_loss: 0.9717 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0530 - accuracy: 0.9762 - val_loss: 1.0933 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0492 - accuracy: 0.9783 - val_loss: 1.2444 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0478 - accuracy: 0.9764 - val_loss: 1.3840 - val_accuracy: 0.7677
Epoch 5/5
24/24 [==============================] - 1s 8ms/step


{'accuracy': 76.64041994750657,
 'precision': 0.7674517658976838,
 'recall': 0.7664041994750657,
 'f1': 0.7645419362124183}

In [17]:
#model5 - 1D CNN
inputs5 = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs5)
x = embedding(x)
x = tf.keras.layers.Conv1D(filters=64,
                           kernel_size=5,
                           padding='same',
                           activation=tf.keras.activations.relu)(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
outputs5 = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)(x)
model_5 = tf.keras.Model(inputs5, outputs5)
model_5.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
history_5 = model_5.fit(train_sentences, train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels))
model_5_preds = model_5.predict(val_sentences)
calculate_results(val_labels, tf.squeeze(tf.round(model_5_preds)))

Epoch 1/5
215/215 [==============================] - 10s 37ms/step - loss: 0.1148 - accuracy: 0.9626 - val_loss: 0.9225 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0699 - accuracy: 0.9743 - val_loss: 1.0034 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0587 - accuracy: 0.9778 - val_loss: 1.0827 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0530 - accuracy: 0.9775 - val_loss: 1.1535 - val_accuracy: 0.7690
Epoch 5/5
24/24 [==============================] - 0s 3ms/step


{'accuracy': 77.03412073490814,
 'precision': 0.7715893693867238,
 'recall': 0.7703412073490814,
 'f1': 0.7684486602580174}

In [18]:
#transfer learning
#model6 - trainsfer learning - USE - Universal sentence encoder
import tensorflow_hub as hub
embed1 = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
sentence_encoder_layer = hub.KerasLayer(embed1,
                                       input_shape=[],
                                       dtype=tf.string,
                                       trainable=False)
model_6 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1, activation = tf.keras.activations.sigmoid)
])
model_6.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
history_6 = model_6.fit(train_sentences, train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels))
model_6_preds=model_6.predict(val_sentences)
calculate_results(val_labels, tf.squeeze(tf.round(model_6_preds)))

Epoch 1/5
215/215 [==============================] - 10s 28ms/step - loss: 0.5090 - accuracy: 0.7815 - val_loss: 0.4482 - val_accuracy: 0.8045
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.4137 - accuracy: 0.8183 - val_loss: 0.4397 - val_accuracy: 0.8110
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.3998 - accuracy: 0.8229 - val_loss: 0.4362 - val_accuracy: 0.8071
Epoch 4/5
215/215 [==============================] - 5s 25ms/step - loss: 0.3923 - accuracy: 0.8273 - val_loss: 0.4282 - val_accuracy: 0.8136
Epoch 5/5
24/24 [==============================] - 1s 19ms/step


{'accuracy': 81.88976377952756,
 'precision': 0.8203089036947304,
 'recall': 0.8188976377952756,
 'f1': 0.8177314483416845}

In [22]:
#model7 - 10%data
# train_df_10_percent = train_df_shuffled[["text","target"]].sample(frac=0.1, random_state=42) - data leakage
train_sentences_10_percent = train_sentences[:int(0.1*len(train_sentences))]
train_labels_10_percent = train_labels[:int(0.1*len(train_labels))]

model_7 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1, activation = tf.keras.activations.sigmoid)
])
model_7.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
history_7 = model_7.fit(train_sentences_10_percent, train_labels_10_percent,
                       epochs=5,
                       validation_data=(val_sentences, val_labels))
model_7_preds=model_7.predict(val_sentences)
calculate_results(val_labels, tf.squeeze(tf.round(model_7_preds)))

Epoch 1/5
22/22 [==============================] - 3s 75ms/step - loss: 0.6798 - accuracy: 0.6336 - val_loss: 0.6550 - val_accuracy: 0.7283
Epoch 2/5
22/22 [==============================] - 1s 48ms/step - loss: 0.6160 - accuracy: 0.8058 - val_loss: 0.6013 - val_accuracy: 0.7625
Epoch 3/5
22/22 [==============================] - 1s 49ms/step - loss: 0.5442 - accuracy: 0.8161 - val_loss: 0.5443 - val_accuracy: 0.7664
Epoch 4/5
22/22 [==============================] - 1s 48ms/step - loss: 0.4809 - accuracy: 0.8248 - val_loss: 0.5116 - val_accuracy: 0.7730
Epoch 5/5
24/24 [==============================] - 1s 19ms/step


{'accuracy': 77.55905511811024,
 'precision': 0.7759863909628747,
 'recall': 0.7755905511811023,
 'f1': 0.7743062301518678}